### This is a WinBUGS Codes for the artificial example in Chapter 9,  Section 9.7.

#### Model: Two-level nonlinear structural equation model

Data Set Name:  YO.dat

Sample Size: N = 1555

Note: pi[g, j] is for omega(2g, j),  lb and lw are for lambda2 and lambda_1,  respectively.

In [2]:
source(".Rprofile")

Loading required package: coda

Loading required package: boot

This is cmdstanr version 0.6.1

- CmdStanR documentation and vignettes: mc-stan.org/cmdstanr

- CmdStan path: /home/pal_bjartan/.cmdstan/cmdstan-2.33.1

- CmdStan version: 2.33.1

This is bayesplot version 1.10.0

- Online documentation and vignettes at mc-stan.org/bayesplot

- bayesplot theme set to bayesplot::theme_default()

   * Does _not_ affect other ggplot2 plots

   * See ?bayesplot_theme_set for details on theme setting



In [5]:
model <- function() {
    for(g in 1:50) {
		for(i in 1:N[g]) { 
			for(j in 1:9) { 
				y[kk[g] + i, j] ~ dnorm(u[kk[g] + i, j], psi[j]) 
				ephat[kk[g] + i, j] <- y[kk[g] + i, j]-u[kk[g] + i, j]
			}
            u[kk[g] + i, 1] <- mu[1] + pi[g, 1] + eta[g, i]
            u[kk[g] + i, 2] <- mu[2] + lb[1] * pi[g, 1] + lw[1] * eta[g, i]
            u[kk[g] + i, 3] <- mu[3] + lb[2] * pi[g, 1] + lw[2] * eta[g, i]
            u[kk[g] + i, 4] <- mu[4] + pi[g, 2] + xi[g, i, 1]
            u[kk[g] + i, 5] <- mu[5] + lb[3] * pi[g, 2] + lw[3] * xi[g, i, 1]
            u[kk[g] + i, 6] <- mu[6] + lb[4] * pi[g, 2] + lw[4] * xi[g, i, 1]
            u[kk[g] + i, 7] <- mu[7] + pi[g, 3] + xi[g, i, 2]
            u[kk[g] + i, 8] <- mu[8] + lb[5] * pi[g, 3] + lw[5] * xi[g, i, 2]
            u[kk[g] + i, 9] <- mu[9] + lb[6] * pi[g, 3] + lw[6] * xi[g, i, 2]
            xi[g, i, 1:2] ~ dmnorm(ux[1:2], phi[1:2, 1:2])      # ux = [0 0]^T  is fixed constant
            eta[g, i] ~ dnorm(nu[g, i],  psd)
            nu[g, i] <- gam[1] * xi[g, i, 1] + gam[2] * xi[g, i, 2] + gam[3] * xi[g, i, 1] * xi[g, i, 2]
			dthat[g, i] <- eta[g, i]-nu[g, i]
		}# end of i
        pi[g, 1:3] ~ dmnorm(uu[1:3], phip[1:3, 1:3])
	}# end of g

    uu[1] <- 0.0
    uu[2] <- 0.0
    uu[3] <- 0.0
    ux[1] <- 0.0
    ux[2] <- 0.0  

    # priors on loadings and coefficients
    mu[1] ~ dnorm(4.248, 4.0)
    mu[2] ~ dnorm(4.668, 4.0)
    mu[3] ~ dnorm(4.56, 4.0)
    mu[5] ~ dnorm(3.161, 4.0)
    mu[6] ~ dnorm(3.445, 4.0)
    mu[7] ~ dnorm(0.526, 4.0)
    mu[8] ~ dnorm(0.375, 4.0)
    mu[9] ~ dnorm(0.596, 4.0)

    var.bw[1] <- 4.0 * psi[2]
    var.bw[2] <- 4.0 * psi[3]
    var.bw[3] <- 4.0 * psi[5]
    var.bw[4] <- 4.0 * psi[6]
    var.bw[5] <- 4.0 * psi[8]
    var.bw[6] <- 4.0 * psi[9]

    lb[1] ~ dnorm(1.096, var.bw[1])
    lb[2] ~ dnorm(0.861, var.bw[2])
    lb[3] ~ dnorm(0.590, var.bw[3])
    lb[4] ~ dnorm(1.470, var.bw[4])
    lb[5] ~ dnorm(0.787, var.bw[5])
    lb[6] ~ dnorm(0.574, var.bw[6])

    lw[1] ~ dnorm(0.825, var.bw[1])
    lw[2] ~ dnorm(0.813, var.bw[2])
    lw[3] ~ dnorm(0.951, var.bw[3])
    lw[4] ~ dnorm(0.692, var.bw[4])
    lw[5] ~ dnorm(0.986, var.bw[5])
    lw[6] ~ dnorm(0.800, var.bw[6])

    var.gam <- 4.0 * psd

    gam[1] ~ dnorm(0.577, var.gam)
    gam[2] ~ dnorm(1.712, var.gam)
    gam[3] ~ dnorm(-0.571, var.gam)

    # priors on precisions
    for(j in 1:9){
        psi[j] ~ dgamma(10.0, 4.0)
        ivpsi[j] <- 1/psi[j]
    }

    psd ~ dgamma(10.0, 4.0)
    ivpsd <- 1/psd

    phi[1:2, 1:2] ~ dwish(R0[1:2, 1:2], 5)
    phx[1:2, 1:2] <- inverse(phi[1:2, 1:2])

    phip[1:3, 1:3] ~ dwish(R1[1:3, 1:3], 5)
    php[1:3, 1:3] <- inverse(phip[1:3, 1:3])

}# end of model

model.path <- glue("{getwd()}/Chapter9/ch9-BUGS-model.txt")
write.model(model, con = model.path)

#### Data


In [8]:
data <- list(
    N = c(28, 27, 25, 28, 33, 26, 18, 26, 17, 24, 26, 24, 24, 30, 23, 24, 29, 
        27, 34, 18, 20, 14, 27, 28, 28, 26, 43, 32, 43, 43, 41, 47, 45, 41, 25, 
        36, 32, 36, 44, 36, 32, 37, 36, 27, 38, 34, 39, 40, 37, 37),
    kk = c(0, 28, 55, 80, 108, 141, 167, 185, 211, 228, 252, 278, 302, 326, 
        356, 379, 403, 432, 459, 493, 511, 531, 545, 572, 600, 628, 654, 697, 
        729, 772, 815, 856, 903, 948, 989, 1014, 1050, 1082, 1118, 1162, 1198, 
        1230, 1267, 1303, 1330, 1368, 1402, 1441, 1481, 1518),
    R0 = structure(
        .Data = c(1.940, 0.775, 0.775, 0.600),
        .Dim = c(2, 2)
    ), 
    R1 = structure(
        .Data = c(13.6, -0.61, 0.48, -0.61, 0.24, 0.06, 0.48, 0.06, 0.22),
        .Dim = c(3, 3)
    ), 
    y = read.csv(
            "./Chapter9/ch9-WinBUGS-data.dat",
            header = FALSE,
            skip = 2
            ) %>%
        .[,1:(ncol(.) - 1)] %>%
        as.matrix() %>%
        unname()
)

#### Three different initial values

In [ ]:
inits <- function() {
    list(
        lb = c(0.6, 0.6, 0.5, 2.2, 0.6, 0.4),
        lw = c(0.3, 0.3, 0.3, 0.3, 0.3, 0.3),
        mu = c(3.0, 3.5, 3.3, 1.0, 2.0, 2.2, 0.2, 0.0, 0.2), 
        psi = c(0.3,  0.3,  0.3,  0.3,  0.3,  0.3, 0.3, 0.3, 0.3),
        psd = 0.6,
        gam = c(0.2, 1.0, -0.4), 
        phip = structure(
            .Data = c(0.7, -0.1, 0.0, -0.1, 0.2, 0.0, 0.0, 0.0, 0.18),
            .Dim = c(3, 3)
        ), 
        phi = structure(
            .Data = c(0.7,  0.4, 0.4, 0.7),
            .Dim = c(2, 2)
        )
    ),

    list(
        lb = c(0.8, 0.8, 0.7, 2.5, 0.8, 0.6),
        lw = c(0.7, 0.7, 0.7, 0.7, 0.7, 0.7),
        mu = c(4.0, 4.0, 4.0, 2.0, 3.0, 3.0, 0.5, 0.4, 0.6),
        psi = c(0.5,  0.5,  0.5,  0.5,  0.5,  0.5, 0.5, 0.5, 0.5),
        psd = 0.36,
        gam = c(0.5, 1.7, 0.6),
        phip = structure(
            .Data = c(0.5, 0.1, -0.1, 0.1, 0.2, 0.0, -0.1, 0.0, 0.5),
            .Dim = c(3, 3)
        ), 
        phi = structure(
            .Data = c(0.5,  0.1, 0.1, 0.5),
            .Dim = c(2, 2)
        )
    ),

    list(
        lb = c(1.0, 1.0, 1.0, 3.0, 1.0, 1.0),
        lw = c(1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
        mu = c(4.8, 4.8, 4.8, 3.5, 4.0, 4.2, 0.8, 0.8, 0.8),
        psi = c(0.8,  0.8,  0.8,  0.8,  0.8,  0.8, 0.8, 0.8, 0.8),
        psd = 0.9,
        gam = c(0.8, 1.2, 0.0), 
        phip = structure(
            .Data = c(0.6, -0.2, 0.2, -0.2, 0.4, 0.1, 0.2, 0.1, 0.3),
            .Dim = c(3, 3)
        ), 
        phi = structure(
            .Data = c(0.9,  0.0, 0.0, 0.6),
            .Dim = c(2, 2)
        )
    )
}

In [17]:
param <- c("lb", "lw", "psi", "gam", "phi", "mu", "xi")
n.iter <- 5000
n.burnin  <- 2000

In [12]:
model.out <- R2WinBUGS::bugs(
    data, 
    inits, 
    param[-length(param)], 
    model.file = model,
    n.chains = 2,
    n.iter = n.iter,
    n.burnin = n.burnin,
    debug = TRUE,
    codaPkg = TRUE,
    bugs.directory = paste0(Sys.getenv("HOME"), "/.wine/drive_c/Program Files (x86)/WinBUGS14/"),
    program = "WinBUGS",
    working.directory = paste0(getwd(), "/Chapter9/bugs-output"),
    WINE = "/usr/bin/wine",
    WINEPATH = "/usr/bin/winepath"
)


ERROR: Error in bugs.run(n.burnin, bugs.directory, WINE = WINE, useWINE = useWINE, : Look at the log file and
try again with 'debug=TRUE' to figure out what went wrong within Bugs.


In [20]:
model.out <- R2OpenBUGS::bugs(
    data, 
    inits, 
    param[-length(param)],  
    n.iter,
    model.file = model.path,
    n.chains = 2,
    n.burnin = n.burnin,
    # debug = TRUE,
    codaPkg = TRUE,
    # OpenBUGS.pgm = paste0(Sys.getenv("HOME"), "/.wine/drive_c/Program Files (x86)/OpenBUGS/OpenBUGS323/OpenBUGS.exe"),
    working.directory = glue("{getwd()}/Chapter9/bugs-output")# ,
    # useWINE = TRUE,
    # WINE = "/usr/bin/wine",
    # WINEPATH = "/usr/bin/winepath"
)


ERROR: Error in bugs.run(n.burnin, OpenBUGS.pgm, debug = debug, WINE = WINE, : Look at the log file in  /home/pal_bjartan/Backup/PhD/SEM-test-model/Lee2007/Chapter9/bugs-output  and
try again with 'debug=TRUE' to figure out what went wrong within OpenBUGS.


In [22]:
rjags::jags.model(
    model.path,
    data,
    inits,
    n.chains = 2,
    n.adapt = n.burnin
)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Deleting model



ERROR: Error in rjags::jags.model(glue("{getwd()}/Chapter9/ch9-BUGS-model.txt"), : RUNTIME ERROR:
Unable to resolve the following parameters:
mu[4] (line 15)
Either supply values for these nodes with the data
or define them on the left hand side of a relation.


